## Sistema de Recomendación


### En este caso utilizaremos la Similaridad del Coseno

Importamos las librerias

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import Tools as t
from sklearn.metrics.pairwise import cosine_similarity
import pyarrow.parquet as pq
import sklearn

Leemos los datos y los guardamos en un dataframe

In [2]:
df_recomendacion = pd.read_csv('data_ok/df_recomendacion.csv', encoding = 'utf-8')

Revisamos la información

In [3]:
df_recomendacion.head()

,user_id,app_name,calificaciones
0,76561197970982479,Killing Floor,5
1,death-hunter,Killing Floor,5
2,DJKamBer,Killing Floor,5
3,diego9031,Killing Floor,5
4,76561198081962345,Killing Floor,4


Creamos una matriz User-Videojuego, para aplicar la Similaridad del Coseno

In [5]:
matriz = df_recomendacion.pivot_table(index='app_name', columns='user_id', values='calificaciones')                     # Hacemos la matriz
matriz.head(20)

user_id,--000--,--ace--,--ionex--,-2SV-vuLB-Kg,-Azsael-,-Beave-,-I_AM_EPIC-,-Kenny,-Mad-,-PRoSlayeR-,...,zuilde,zukuta,zunbae,zuzuga2003,zv_odd,zvanik,zwanzigdrei,zy0705,zynxgameth,zyr0n1c
app_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
//N.P.P.D. RUSH//- The milk of Ultraviolet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0RBITALIS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"10,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100% Orange Juice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100% Orange Juice - Krila & Kae Character Pack,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001 Spikes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12 Labours of Hercules,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12 Labours of Hercules II: The Cretan Bull,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Normalizamos los valores de calificación restandoas con su promedio, para que quede en un rango de valores más equitativo 

In [6]:
matriz_normal = matriz.subtract(matriz.mean(axis=1), axis = 'rows')             
matriz_normal.head()

user_id,--000--,--ace--,--ionex--,-2SV-vuLB-Kg,-Azsael-,-Beave-,-I_AM_EPIC-,-Kenny,-Mad-,-PRoSlayeR-,...,zuilde,zukuta,zunbae,zuzuga2003,zv_odd,zvanik,zwanzigdrei,zy0705,zynxgameth,zyr0n1c
app_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
//N.P.P.D. RUSH//- The milk of Ultraviolet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0RBITALIS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"10,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100% Orange Juice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Rellenamos todos los valores nulos con 0, de lo contrario no podremos utilizar similarity coseno

In [7]:
matriz_normal = matriz_normal.fillna(0)                                                                      # Rellenamos con 0

matriz_normal = matriz_normal.loc[:, (matriz_normal != 0).any(axis=0)]                                       # Eliminamos los elementos completamente nulos

item_similarity_coseno = cosine_similarity(matriz_normal)                                                    # Aplicamos la similaridad del coseno
item_similarity_coseno

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Para mayor facilidad y visualización, pasamos el array item_similarity_coseno a un dataframe de pandas

In [8]:
df_i_sim_cos= pd.DataFrame(item_similarity_coseno, index = matriz_normal.index, columns = matriz_normal.index)
df_i_sim_cos

app_name,! That Bastard Is Trying To Steal Our Gold !,//N.P.P.D. RUSH//- The milk of Ultraviolet,0RBITALIS,"10,000,000",100% Orange Juice,100% Orange Juice - Krila & Kae Character Pack,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,...,liteCam Game: 100 FPS Game Capture,nail'd,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theHunter Classic,theHunter: Primal,Астролорды: Облако Оорта
app_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
//N.P.P.D. RUSH//- The milk of Ultraviolet,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0RBITALIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"10,000,000",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100% Orange Juice,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sZone-Online,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
theHunter Classic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


Guardamos el dataframe en un archivo parquet para su posterior análisis

In [9]:
df_i_sim_cos.to_parquet('datos_parquet/df_recomendacion_juego.parquet')

### Funcion

In [10]:
def recomendacion_juego(juego):

    similares = df_i_sim_cos.sort_values(by=juego, ascending=False).index[1:6]                               # Acomoda los valores de mayor a menor, y toma sólo los primeros 5
    print(f'Los 5 juegos más similares a {juego} son:\n')
    for count, item in enumerate(similares, start=1):                                                        # Itera sobre el dataframe similares para imprimir los resultados
        print(f'No. {count}: {item}')

Vamos a probar la función

In [11]:
top = recomendacion_juego('100% Orange Juice')

Los 5 juegos más similares a 100% Orange Juice son:

No. 1: Planet Explorers
No. 2: Talisman: Digital Edition
No. 3: Lethal League
No. 4: Danganronpa: Trigger Happy Havoc
No. 5: Eryi's Action


In [12]:
top = recomendacion_juego('theHunter Classic')

Los 5 juegos más similares a theHunter Classic son:

No. 1: Galactic Civilizations III
No. 2: Please, Don’t Touch Anything
No. 3: Card Hunter
No. 4: Floating Point
No. 5: Bloons TD Battles


 <h2 align=center> ______________________________________________________________________<h2>

Este es el fin del procesamiento de los datasets. Puede consultar el archivo [main.py](main.py) para observar las funciones que se deployarán, o bien el archivo [requirements.txt](requirements.txt) que fue utilizado para hacer el deploy del API en render.

Después de este punto puede acceder [aqui](https://steam-pi-4c9l.onrender.com/docs#/) para probar las funciones previamente mencionadas en estos notebooks.

O bien puede acceder al video resumen del proyecto [aqui](https://www.youtube.com/watch?v=3FRkXWo55Yw).

Gracias.